In [2]:
import pickle
import numpy as np
import torch
from tqdm import tqdm
import numqi
import matplotlib.pyplot as plt

In [3]:
# Random generate a d^-1-dimensional real vector in Euclidean space
def random_vector(d):
    x = np.random.normal(size=d*d-1)
    x = x / np.linalg.norm(x)
    return x

def save_data(xdata, ydata, filepath='data_boundary.pkl'):
    with open(filepath, 'wb') as fid:
        pickle.dump(dict(xdata=xdata, ydata=ydata), fid)

def load_data(filepath='data_boundary.pkl'):
    with open(filepath, 'rb') as fid:
        tmp0 = pickle.load(fid)
        ret = tmp0['xdata'], tmp0['ydata']
    return ret

In [5]:
dimA = 3
dimB = 3 
dim = dimA * dimB
num_sample_random_mixed = 1000000
num_sample_random_pure = int(num_sample_random_mixed // 4)
num_sample = num_sample_random_mixed + num_sample_random_pure
batch_size = 128

In [7]:
print("random mixed state")
vec = random_vector(dim)
rho = numqi.gellmann.gellmann_basis_to_dm(vec)
model = numqi.entangle.AutodiffCHAREE(dimA, dimB, distance_kind='ree')
beta_cha = model.get_boundary(rho, use_tqdm=False)
_, beta_ppt = numqi.entangle.get_ppt_boundary(rho,(dimA,dimB))
print('beta_cha =', beta_cha)
print('beta_ppt =', beta_ppt)

print("random pure state")
state = numqi.random.rand_bipartite_state(dimA, dimB)
rho = state[:, np.newaxis] @ state[np.newaxis, :].conj()
model = numqi.entangle.AutodiffCHAREE(dimA, dimB, distance_kind='ree')
beta_cha = model.get_boundary(rho, use_tqdm=False)
_, beta_ppt = numqi.entangle.get_ppt_boundary(rho,(dimA,dimB))
print('beta_cha =', beta_cha)
print('beta_ppt =', beta_ppt)

random mixed state
beta_cha = 0.15573127699836403
beta_ppt = 0.15580735480832905
random pure state
beta_cha = 0.14786783854166657
beta_ppt = 0.14776371779285896


In [ ]:
# Training data
model = numqi.entangle.AutodiffCHAREE(dimA, dimB, distance_kind='ree')
xdata = []
ydata = []
vec_list = [random_vector(dim) for _ in range(num_sample_random_mixed)]
for vec in tqdm(vec_list):
    rho = numqi.gellmann.gellmann_basis_to_dm(vec)
    beta = model.get_boundary(rho, use_tqdm=False)
    xdata.append(vec)
    ydata.append(beta)
print(f'Mixed ppt state: min: {np.min(ydata)}, max: {np.max(ydata)}, mean: {np.mean(ydata)}, variance: {np.var(ydata)}')
save_data(xdata, ydata, f'data_sep_mixed_boundary_{dimA}*{dimB}_{num_sample_random_mixed}.pkl') 

xdata = []
ydata = []
pure_state_list = [numqi.random.rand_bipartite_state(dimA, dimB) for _ in range(num_sample_random_pure)]
for state in tqdm(pure_state_list):
    rho = state[:, np.newaxis] @ state[np.newaxis, :].conj()
    beta = model.get_boundary(rho, use_tqdm=False)
    vec = numqi.gellmann.dm_to_gellmann_basis(rho)
    xdata.append(vec)
    ydata.append(beta)
xdata = np.array(xdata)
ydata = np.array(ydata)
print(f'Pure ppt state: min: {np.min(ydata)}, max: {np.max(ydata)}, mean: {np.mean(ydata)}, variance: {np.var(ydata)}')
save_data(xdata, ydata, f'data_sep_pure_boundary_{dimA}*{dimB}_{num_sample_random_pure}.pkl') 

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()
        tmp0 = [dim**2-1, 128, 256, 256, 512, 512, 512, 256, 256, 128, 1]
        self.fc_list = torch.nn.ModuleList([torch.nn.Linear(tmp0[x], tmp0[x+1]) for x in range(len(tmp0)-1)])
        self.bn_list = torch.nn.ModuleList([torch.nn.BatchNorm1d(tmp0[x+1]) for x in range(len(tmp0)-1)])
        self.a = torch.sqrt(torch.tensor(1/(2*dim*(dim-1)), dtype=torch.float32))
        self.b = torch.sqrt(torch.tensor((dim-1)/(2*dim), dtype=torch.float32))
        self.scale = torch.tensor(0.05, dtype=torch.float32)

    def forward(self, x):
        for ind0 in range(len(self.fc_list)):
            y = self.fc_list[ind0](x)
            y = self.bn_list[ind0](y)
            if ind0<(len(self.fc_list)-1):
                y = torch.nn.functional.leaky_relu(y)
            if x.shape[-1]==y.shape[-1]:
                x = y + x
            else:
                x = y
        x = (1 + self.scale) * (self.b - self.a) * torch.sigmoid(x) + self.a * (1 - self.scale)
        x = x[:,0] 
        return x

In [ ]:
xdata_mixed,ydata_mixed = load_data(filepath=f'data_ppt_mixed_boundary_{dimA}*{dimB}_{num_sample_random_mixed}.pkl')
xdata_pure,ydata_pure = load_data(filepath=f'data_ppt_pure_boundary_{dimA}*{dimB}_{num_sample_random_pure}.pkl')
xdata = np.concatenate([xdata_mixed, xdata_pure], axis=0)
ydata = np.concatenate([ydata_mixed, ydata_pure], axis=0)
tmp0 = torch.tensor(xdata, dtype=torch.float32)
tmp1 = torch.tensor(ydata, dtype=torch.float32)
dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(tmp0, tmp1), batch_size=batch_size, shuffle=True)

In [ ]:
model = Net(dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_history = []
model.train()
for epoch in range(1):
    with tqdm(dataloader) as pbar:
        for x, y in pbar:
            optimizer.zero_grad()
            y_pred = model(x)
            loss = torch.nn.functional.mse_loss(y_pred, y)
            loss.backward()
            optimizer.step()
            loss_history.append(loss.item())
            pbar.set_postfix(train_loss='{:.5}'.format(sum(loss_history[-10:])/10))

In [ ]:
fig,ax = plt.subplots()
ax.plot(np.arange(len(loss_history)), loss_history, '.')
ax.set_xlabel('iteration')
ax.set_ylabel('loss')
# log scale
ax.set_yscale('log')
ax.grid()
fig.tight_layout()